In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
pd.options.display.float_format = '{:.2f}'.format

from itertools import combinations
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from prophet import Prophet

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Measles/nigeria.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Year-Month  159 non-null    object
 1   Cases       159 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.6+ KB


In [5]:
df.columns = ['ds','y']

In [6]:
train_data_pr = df.iloc[:len(df)-12]
test_data_pr = df.iloc[len(df)-12:]

In [7]:
len(test_data_pr)

12

In [8]:
test_data_pr['y']

147    1747
148    1598
149    1113
150     613
151     321
152     196
153     245
154     200
155     134
156    2132
157     696
158      12
Name: y, dtype: int64

In [9]:
from enum import auto
#m = Prophet()
m = Prophet(seasonality_mode = "multiplicative",growth='linear',yearly_seasonality=True,weekly_seasonality=False,daily_seasonality=False,
            changepoints=None, changepoint_range=0)
m.fit(train_data_pr)
future = m.make_future_dataframe(periods=12,freq='MS')
prophet_pred = m.predict(future)

INFO:prophet:n_changepoints greater than number of observations. Using -1.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpk_aww2jy/slpdiho3.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpk_aww2jy/n7_qpmt0.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=1337', 'data', 'file=/tmp/tmpk_aww2jy/slpdiho3.json', 'init=/tmp/tmpk_aww2jy/n7_qpmt0.json', 'output', 'file=/tmp/tmpk_aww2jy/prophet_modeljh3obxe2/prophet_model-20240720075218.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
07:52:18 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:52:18 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
07:52:18 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
ERROR:cmdstanpy:Chain [1] error: error during processing Operation not p

In [10]:
prophet_pred = pd.DataFrame({"Date" : prophet_pred[-12:]['ds'], "Pred" : prophet_pred[-12:]["yhat"]})

In [11]:
prophet_pred = prophet_pred.set_index("Date")

In [12]:
prophet_pred.index.freq = "MS"

In [13]:
prophet_pred

,Pred
Date,
2023-04-01,1898.03
2023-05-01,1004.69
2023-06-01,429.36
2023-07-01,225.92
2023-08-01,162.33
2023-09-01,196.17
2023-10-01,320.75
2023-11-01,418.89
2023-12-01,349.78


In [15]:
from sklearn.metrics import mean_squared_error
error = np.sqrt(mean_squared_error(test_data_pr['y'],prophet_pred['Pred']))
#print('Test RMSE: %.4f' % error)

def mean_absolute_percentage_error(y_true, y_pred):

    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


mape = mean_absolute_percentage_error(test_data_pr['y'],prophet_pred['Pred'])

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(test_data_pr['y'],prophet_pred['Pred'])
#print("Mean Absolute Error (MAE):", mae)

from sklearn.metrics import mean_squared_log_error
msle = mean_squared_log_error(test_data_pr['y'],prophet_pred['Pred'])
#print("Mean Squared Logarithmic Error (MSLE):", msle)

from sklearn.metrics import mean_absolute_percentage_error
#print('Mean Absolute Percentage Error (MAPE):',mean_absolute_percentage_error(test_data_pr['y'],prophet_pred['Pred']))

print('RMSE: %.2f' % error)
print(f"MAPE: {mape:.2f}%")
print('MAE:  %.2f' % mae)
print('MSLE: %.2f' % msle)

RMSE: 826.52
MAPE: 1699.92%
MAE:  546.58
MSLE: 2.68
